In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,col, to_date

In [14]:
!pip install prettytable

In [1]:

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate() 

In [28]:
df = spark.read.format("json").load("data/").select("tracks")

In [47]:
df.printSchema()

root
 |-- tracks: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- items: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- added_at: string (nullable = true)
 |    |    |    |-- added_by: struct (nullable = true)
 |    |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- uri: string (nullable = true)
 |    |    |    |-- is_local: boolean (nullable = true)
 |    |    |    |-- primary_color: string (nullable = true)
 |    |    |    |-- track: struct (nullable = true)
 |    |    |    |    |-- album: struct (nullable = true)
 |    |    |    |    |    |-- album_type: string (nullable = true)
 |    |    |    |    |    |-- artists: array (nullable = true)
 |    |

In [57]:
# df_data = df.withColumn("items",explode("tracks.items")).select("items")
df_data = df.withColumn("tracks_items",col("tracks.items")).select("tracks_items")


In [58]:
df_data.printSchema()

root
 |-- tracks_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- added_at: string (nullable = true)
 |    |    |-- added_by: struct (nullable = true)
 |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- uri: string (nullable = true)
 |    |    |-- is_local: boolean (nullable = true)
 |    |    |-- primary_color: string (nullable = true)
 |    |    |-- track: struct (nullable = true)
 |    |    |    |-- album: struct (nullable = true)
 |    |    |    |    |-- album_type: string (nullable = true)
 |    |    |    |    |-- artists: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    

In [59]:
df_items = df_data.withColumn("items",explode("tracks_items")).select("items")

In [67]:
df_items.printSchema()

root
 |-- items: struct (nullable = true)
 |    |-- added_at: string (nullable = true)
 |    |-- added_by: struct (nullable = true)
 |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |-- spotify: string (nullable = true)
 |    |    |-- href: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- uri: string (nullable = true)
 |    |-- is_local: boolean (nullable = true)
 |    |-- primary_color: string (nullable = true)
 |    |-- track: struct (nullable = true)
 |    |    |-- album: struct (nullable = true)
 |    |    |    |-- album_type: string (nullable = true)
 |    |    |    |-- artists: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- external_urls: struct (nullable = true)
 |    |    |    |    |    |    |-- spotify: string (nullable = true)
 |    |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |   

In [78]:
# Album df
df_album = df_items.withColumn("album_id",col("items.track.album.id"))\
            .withColumn("album_name",col("items.track.album.name"))\
            .withColumn("album_release_date",col("items.track.album.release_date"))\
            .withColumn("album_total_tracks",col("items.track.album.total_tracks"))\
            .withColumn("album_url",col("items.track.album.external_urls.spotify"))\
            .select("album_id","album_name","album_release_date","album_total_tracks","album_url")

In [79]:
df_album.show(truncate=False)

+----------------------+---------------------------------------------------------+------------------+------------------+-----------------------------------------------------+
|album_id              |album_name                                               |album_release_date|album_total_tracks|album_url                                            |
+----------------------+---------------------------------------------------------+------------------+------------------+-----------------------------------------------------+
|75SlrAXlLbJN9LYJwlTJh6|Mismatched: Season 3 (Soundtrack from the Netflix Series)|2024-12-03        |12                |https://open.spotify.com/album/75SlrAXlLbJN9LYJwlTJh6|
|2yYfIOq25JQWvUQ9AR172D|Sanam Teri Kasam (Original Motion Picture Soundtrack)    |2016-01-06        |9                 |https://open.spotify.com/album/2yYfIOq25JQWvUQ9AR172D|
|1poClftP5T3zRcqpsxPPfW|Raanjhan (From "Do Patti")                               |2024-10-04        |1                 |https

In [90]:
# Songs df
df_songs = df_items.withColumn("song_id",col("items.track.id"))\
            .withColumn("song_name",col("items.track.name"))\
            .withColumn("song_duration",col("items.track.duration_ms"))\
            .withColumn("song_url",col("items.track.external_urls.spotify"))\
            .withColumn("song_popularity",col("items.track.popularity"))\
            .withColumn("song_added",col("items.added_at"))\
            .withColumn("album_id",col("items.track.album.id"))\
            .withColumn("artist_id",col("items.track.album.artists").getItem(0).getItem("id"))\
            .select("song_id","song_name","song_duration","song_url","song_popularity","song_added","album_id","artist_id")

            # .withColumn("artist_id",col("items.track.album.external_urls.spotify"))\

In [91]:
df_songs.show(truncate=False)

+----------------------+--------------------------------------------+-------------+-----------------------------------------------------+---------------+--------------------+----------------------+----------------------+
|song_id               |song_name                                   |song_duration|song_url                                             |song_popularity|song_added          |album_id              |artist_id             |
+----------------------+--------------------------------------------+-------------+-----------------------------------------------------+---------------+--------------------+----------------------+----------------------+
|3vCzLB6kS2lGcIpm1OOUsy|Ishq Hai                                    |312857       |https://open.spotify.com/track/3vCzLB6kS2lGcIpm1OOUsy|78             |2024-12-21T06:12:39Z|75SlrAXlLbJN9LYJwlTJh6|0NbgKDsEHiPzZBZkqZGkXm|
|1PDREqVNvmphIoNac3Vz8c|Sanam Teri Kasam                            |314000       |https://open.spotify.com/track/1P

In [104]:
# Artist df
df_artists_temp = df_items.withColumn("artist_data",explode(col("items.track.artists")))
df_artists = df_artists_temp.withColumn("artist_id",col("artist_data.id"))\
                            .withColumn("artist_name",col("artist_data.name")) \
                            .withColumn("external_url",col("artist_data.href"))
df_artists.select("artist_data")

DataFrame[artist_data: struct<external_urls:struct<spotify:string>,href:string,id:string,name:string,type:string,uri:string>]